In [1]:
!gdown --id 18qSvsM5W2QFASUlLWFPqxuYFPZcOs25b

Downloading...
From: https://drive.google.com/uc?id=18qSvsM5W2QFASUlLWFPqxuYFPZcOs25b
To: /content/LUMS_FALL2020_PROJECT_DATA1.zip
802MB [00:08, 98.8MB/s]


In [2]:
!unzip "LUMS_FALL2020_PROJECT_DATA1.zip"

Archive:  LUMS_FALL2020_PROJECT_DATA1.zip
   creating: Gender_Recognition/Test/
   creating: Gender_Recognition/Test/SPK083_M/
  inflating: Gender_Recognition/Test/SPK083_M/1.wav  
  inflating: Gender_Recognition/Test/SPK083_M/10.wav  
  inflating: Gender_Recognition/Test/SPK083_M/2.wav  
  inflating: Gender_Recognition/Test/SPK083_M/3.wav  
  inflating: Gender_Recognition/Test/SPK083_M/4.wav  
  inflating: Gender_Recognition/Test/SPK083_M/5.wav  
  inflating: Gender_Recognition/Test/SPK083_M/6.wav  
  inflating: Gender_Recognition/Test/SPK083_M/7.wav  
  inflating: Gender_Recognition/Test/SPK083_M/8.wav  
  inflating: Gender_Recognition/Test/SPK083_M/9.wav  
   creating: Gender_Recognition/Test/SPK084_M/
  inflating: Gender_Recognition/Test/SPK084_M/1.wav  
  inflating: Gender_Recognition/Test/SPK084_M/10.wav  
  inflating: Gender_Recognition/Test/SPK084_M/2.wav  
  inflating: Gender_Recognition/Test/SPK084_M/3.wav  
  inflating: Gender_Recognition/Test/SPK084_M/4.wav  
  inflating: G

# Machine Learning Project
Will be used to make the Speaker Recognition Part

In [3]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=9b815dd28aca93ac6eaeb39602456494c28b781ac3450e67ca63bcd8736314e1
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import python_speech_features as mfcc
from scipy.io.wavfile import read
import pickle
import glob
import time
import os
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

### Shared Functions

In [13]:
def get_MFCC(audio, sr):
    features = mfcc.mfcc(audio, sr, 0.025, 0.01, 13, appendEnergy = True)
    return np.mean(features, axis=0)

def get_numpy(elem):
  return np.array(elem)

def final_accuracies(y_test, y_pred):
  acc_score = (metrics.accuracy_score(y_test, y_pred))
  acc_score = (round(acc_score,2)) * 100

  macro_precision = (metrics.precision_score(y_test, y_pred, average='macro'))
  macro_precision = (round(macro_precision,2)) * 100

  macro_recall = (metrics.recall_score(y_test, y_pred, average='macro'))
  macro_recall = (round(macro_recall,2)) * 100

  macro_f1 = (metrics.f1_score(y_test, y_pred, average='macro'))
  macro_f1 = (round(macro_f1,2))*100

  confusion = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
  creport = classification_report(y_test, y_pred)

  return acc_score, macro_precision, macro_recall, macro_f1, confusion, creport

# Gender Recognition

In [ ]:
def get_features2(directory):
  features_list = []
  labels_list = []
  for file in glob.iglob(directory):
    if 'F' in file:
      labels_list.append(0)
      sr, audio = read(file)
      features = get_MFCC(audio, sr)
      features_list.append(features)
    elif 'M' in file:
      labels_list.append(1)
      sr, audio = read(file)
      features = get_MFCC(audio, sr)
      features_list.append(features)
      
  return features_list, labels_list

def one_hot_converter(labels,y):
    one_hot = []
    for i in range(len(y)):
        vec = np.zeros((len(labels,)))
        vec[y[i][0]] = 1
        one_hot.append(vec)
    return np.array(one_hot)

In [ ]:
starttime = time.time() 
X_train, y_train = get_features2("Gender_Recognition/Train/**/*.wav")
print ("Training Reading Time is = ", str(time.time() - starttime) + " seconds")

starttime = time.time() 
X_test, y_test = get_features2("Gender_Recognition/Test/**/*.wav")
print ("TestingReading Time is = ", str(time.time() - starttime) + " seconds")

Training Reading Time is =  46.916107177734375 seconds
TestingReading Time is =  5.806235074996948 seconds


In [ ]:
X_train = get_numpy(X_train)
y_train = get_numpy(y_train)
X_test = get_numpy(X_test)
y_test = get_numpy(y_test)

In [ ]:
X_train = np.concatenate([np.ones((len(X_train), 1)), X_train], axis=1)
X_test = np.concatenate([np.ones((len(X_test), 1)), X_test], axis=1)

In [ ]:
print("Training Data X", X_train.shape)
print("Testing Data X", X_test.shape)
print()
print("Training Data Y", y_train.shape)
print("Testing Data Y", y_test.shape)

Training Data X (1250, 14)
Testing Data X (170, 14)

Training Data Y (1250,)
Testing Data Y (170,)


In [ ]:
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

In [ ]:
print("Training Data Y", y_train.shape)
print("Testing Data Y", y_test.shape)

Training Data Y (1250, 1)
Testing Data Y (170, 1)


In [ ]:
unique_labels = np.unique(y_train)
unique_labels

array([0, 1])

###Part 1 - Multi-Layer Perceptron

In [ ]:
model = MLPClassifier(random_state=1, max_iter=5000, activation='logistic', solver='sgd')
parameters = {'hidden_layer_sizes': [(128,64),(64),(64,32),(32)], 'learning_rate_init': [0.4,0.1,0.01]}
clf3 = GridSearchCV(model, parameters)
clf3.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d

GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='logistic', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=5000, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=1, shuffle=True, solver='sgd',
                                     tol=0.0001, validation_fraction=0.1,
                                     verbose=False, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'hidden_layer_sizes': [(128, 64)

In [ ]:
print(clf3.best_params_)

{'hidden_layer_sizes': (128, 64), 'learning_rate_init': 0.4}


In [ ]:
y_pred = clf3.predict(X_test)
y_pred

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
acc_score, macro_precision, macro_recall, macro_f1, confusion, classifcation = final_accuracies(y_test, y_pred)

print("Accuracy Score is: {}%".format(acc_score))
print("Macroaveraged Recall is: {}%".format(macro_recall))
print("Macroaveraged Precision is: {}%".format(macro_precision))
print("Macroaveraged F1-score is: {}%".format(macro_f1))

Accuracy Score is: 88.0%
Macroaveraged Recall is: 82.0%
Macroaveraged Precision is: 83.0%
Macroaveraged F1-score is: 82.0%


In [ ]:
print("\nConfusion Matrix :\n")
print(confusion)


Confusion Matrix :

[[ 28  12]
 [  9 121]]


In [ ]:
print("\nClassification Report:\n")
print(classifcation)


Classification Report:

              precision    recall  f1-score   support

           0       0.76      0.70      0.73        40
           1       0.91      0.93      0.92       130

    accuracy                           0.88       170
   macro avg       0.83      0.82      0.82       170
weighted avg       0.87      0.88      0.87       170



###Part 2 - SVM

In [ ]:
clf2 = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf2.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=0,
                           tol=1e-05, verbose=0))],
         verbose=False)

In [ ]:
y_pred = clf2.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
acc_score, macro_precision, macro_recall, macro_f1, confusion, classifcation = final_accuracies(y_test, y_pred)

print("Accuracy Score is: {}%".format(acc_score))
print("Macroaveraged Recall is: {}%".format(macro_recall))
print("Macroaveraged Precision is: {}%".format(macro_precision))
print("Macroaveraged F1-score is: {}%".format(macro_f1))

Accuracy Score is: 84.0%
Macroaveraged Recall is: 74.0%
Macroaveraged Precision is: 78.0%
Macroaveraged F1-score is: 75.0%


In [ ]:
print("\nConfusion Matrix :\n")
print(confusion)


Confusion Matrix :

[[ 22  18]
 [ 10 120]]


In [ ]:
print("\nClassification Report:\n")
print(classifcation)


Classification Report:

              precision    recall  f1-score   support

           0       0.69      0.55      0.61        40
           1       0.87      0.92      0.90       130

    accuracy                           0.84       170
   macro avg       0.78      0.74      0.75       170
weighted avg       0.83      0.84      0.83       170



###Part 3 - Gaussian

In [ ]:
clf1 = GaussianNB()
clf1.fit(X_train,y_train.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_pred = clf1.predict(X_test)
y_pred

array([0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
acc_score, macro_precision, macro_recall, macro_f1, confusion, classifcation = final_accuracies(y_test, y_pred)

print("Accuracy Score is: {}%".format(acc_score))
print("Macroaveraged Recall is: {}%".format(macro_recall))
print("Macroaveraged Precision is: {}%".format(macro_precision))
print("Macroaveraged F1-score is: {}%".format(macro_f1))

Accuracy Score is: 85.0%
Macroaveraged Recall is: 77.0%
Macroaveraged Precision is: 80.0%
Macroaveraged F1-score is: 79.0%


In [ ]:
print("\nConfusion Matrix :\n")
print(confusion)


Confusion Matrix :

[[ 25  15]
 [ 10 120]]


In [ ]:
print("\nClassification Report:\n")
print(classifcation)


Classification Report:

              precision    recall  f1-score   support

           0       0.71      0.62      0.67        40
           1       0.89      0.92      0.91       130

    accuracy                           0.85       170
   macro avg       0.80      0.77      0.79       170
weighted avg       0.85      0.85      0.85       170



#Speaker Recognition

In [6]:
def extract_label(file):
  r = file.split('SPK')[-1]
  return r[:3]

def get_features(directory):
  features_list = []
  labels_list = []
  for file in glob.iglob(directory):
    extracted_label = extract_label(file)
    labels_list.append(extracted_label)
    sr, audio = read(file)
    features = get_MFCC(audio, sr)
    features_list.append(features)
  return get_array(features_list, labels_list)

def get_array(features, labels):
  m = len(features)
  X = np.concatenate([np.ones((m, 1)), features], axis=1)
  X, Y = np.array(X), np.array(list(map(int, labels)))
  return X, Y.reshape(Y.shape[0], 1)

def one_hot_converter2(labels,y):
    one_hot = []
    for i in range(len(y)):
        vec = np.zeros((len(labels,)))
        vec[y[i][0]] = 1
        one_hot.append(vec)
    return np.array(one_hot)

In [7]:
starttime = time.time() 
X_train_speaker, y_train_speaker = get_features("Speaker_Recognition/Train/**/*.wav")
print ("Training Reading Time is = ", str(time.time() - starttime) + " seconds")

starttime = time.time() 
X_test_speaker, y_test_speaker = get_features("Speaker_Recognition/Test/**/*.wav")
print ("TestingReading Time is = ", str(time.time() - starttime) + " seconds")

Training Reading Time is =  38.636030435562134 seconds
TestingReading Time is =  9.776883602142334 seconds


In [8]:
print("Training Data X", X_train_speaker.shape)
print("Testing Data X", X_test_speaker.shape)
print()
print("Training Data Y", y_train_speaker.shape)
print("Testing Data Y", y_test_speaker.shape)

Training Data X (1136, 14)
Testing Data X (284, 14)

Training Data Y (1136, 1)
Testing Data Y (284, 1)


In [9]:
unique_labels_speaker = np.unique(y_train_speaker)
unique_labels_speaker.shape

(142,)

###Part 1 - Multi-Layer Perceptron

In [17]:
model = MLPClassifier(random_state=1, max_iter=5000, activation='logistic', solver='sgd')
parameters = {'hidden_layer_sizes': [(128,64),(64),(64,32),(32)], 'learning_rate_init': [0.4,0.1,0.01]}
clf3 = GridSearchCV(model, parameters)
clf3.fit(X_train_speaker, y_train_speaker)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d

GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='logistic', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=5000, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=1, shuffle=True, solver='sgd',
                                     tol=0.0001, validation_fraction=0.1,
                                     verbose=False, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'hidden_layer_sizes': [(128, 64)

In [18]:
print(clf3.best_params_)

{'hidden_layer_sizes': 64, 'learning_rate_init': 0.01}


In [19]:
y_pred_speaker = clf3.predict(X_test_speaker)
y_pred_speaker

array([ 16,  16,  48,  48, 126, 126, 142, 142,  79,  79, 133, 133,   8,
         8,  84,  84, 106, 106,  66,  66,  68,  68,  34,  34,  22,  22,
        37,  37, 138, 138,  73,  73, 135, 135,  77,  77, 104, 104,  28,
        28,  88, 134,   5,   5, 116, 116, 121, 121, 139, 139,  17,  17,
        59,  59,  10,  10,  47,  47,  11,  11,  70,  70,  13,  13, 114,
       114,  91,  52, 102, 102,  81,  81,  18,  18,  65,  85,  71,  71,
        90,  96, 136, 136, 141, 141, 113, 113,  99,  99,  41,  41, 111,
        49,  83,  83,  23,  30,  35,  35,  38,  38, 103, 103,  64,  64,
        76,  96,  43,  43,  53,  53,  45,  45,  44,  44,  87,  87,  94,
        94,  65,  65,  26,  26,   1,   1, 107, 107, 111, 111,  25,  25,
       137, 137, 105, 105,  12,  12,  33,  33,  54,  54,  69,  69,   9,
         9,  75,  75,  60,  60,  20, 139, 110, 110, 108, 108,  57,  57,
        61,  61,  32,  74,  78,  78,  59,  32, 140, 140, 124, 124, 123,
       123,  23, 101,  72,  72,  24,  24,  67,  67,  55,  55,  7

In [20]:
acc_score, macro_precision, macro_recall, macro_f1, confusion, classifcation = final_accuracies(y_test_speaker, y_pred_speaker)

print("Accuracy Score is: {}%".format(acc_score))
print("Macroaveraged Recall is: {}%".format(macro_recall))
print("Macroaveraged Precision is: {}%".format(macro_precision))
print("Macroaveraged F1-score is: {}%".format(macro_f1))

Accuracy Score is: 95.0%
Macroaveraged Recall is: 95.0%
Macroaveraged Precision is: 96.0%
Macroaveraged F1-score is: 95.0%


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
print("\nConfusion Matrix :\n")
print(confusion)


Confusion Matrix :

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]]


In [22]:
print("\nClassification Report:\n")
print(classifcation)


Classification Report:

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         2
          16       1.00      1.00      1.00         2
          17       1.00      1.00      1.00         2
  

###Part 2 - SVM

In [10]:
clf2 = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf2.fit(X_train_speaker, y_train_speaker)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=0,
                           tol=1e-05, verbose=0))],
         verbose=False)

In [11]:
y_pred_speaker = clf2.predict(X_test_speaker)
y_pred_speaker

array([ 16,  16,  48,  48, 126, 126, 142, 142,  79,  79, 133, 133,   8,
         8,  84,  84, 106, 106,  66,  66,  68,  68,  34,  34,  22,  22,
        37,  37, 138, 138,  73,  73, 135, 135,  77,  77, 104, 104,  28,
        28, 134, 134,   5,   5, 116, 116, 121, 121, 139, 139,  10,  17,
        59,  59,  10,  10,  47,  47,  11,  11,  70,  70,  13,  13, 114,
       114,  91, 119, 102, 102,  81,  81,  18,  18,  65,  85,  71,  71,
        90,  90, 136, 136, 141, 141, 113, 113,   7,  99,  41,  41, 111,
        49,  83,  83,  30,  30, 137,  35,  38,  79,  17, 103,  64,  64,
        76,  96,  43,  43,  53,  53,  45,  95,  44,  44,  87,  87,  94,
        94,  65,  65,  26,  26,   1,   1, 107, 107, 111, 111,  25,  25,
       137, 137, 105, 105,  12,  12,  33,  33,  54,  54,  69,  69,   9,
         9,  75,  75,  60,  60,  20,  19, 110, 110, 108, 108,  57,  57,
        61,  61,  74,  74,  78,  78, 119,  38, 140, 140, 124, 124, 129,
       123,  83, 101,  72,  72,   3,  24,  67,  67,  55,  55,  7

In [14]:
acc_score, macro_precision, macro_recall, macro_f1, confusion, classifcation = final_accuracies(y_test_speaker, y_pred_speaker)

print("Accuracy Score is: {}%".format(acc_score))
print("Macroaveraged Recall is: {}%".format(macro_recall))
print("Macroaveraged Precision is: {}%".format(macro_precision))
print("Macroaveraged F1-score is: {}%".format(macro_f1))

Accuracy Score is: 93.0%
Macroaveraged Recall is: 93.0%
Macroaveraged Precision is: 94.0%
Macroaveraged F1-score is: 93.0%


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
print("\nConfusion Matrix :\n")
print(confusion)


Confusion Matrix :

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]]


In [16]:
print("\nClassification Report:\n")
print(classifcation)


Classification Report:

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       0.67      1.00      0.80         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       0.50      1.00      0.67         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       0.67      1.00      0.80         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         2
          16       1.00      1.00      1.00         2
          17       0.50      0.50      0.50         2
  

###Part 3 - Gaussian

In [23]:
clf3 = GaussianNB()
clf3.fit(X_train_speaker,y_train_speaker)

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
y_pred_speaker = clf3.predict(X_test_speaker)
y_pred_speaker

array([ 16,  16,  48,  48, 126, 126, 142, 142,  79,  79, 133, 133,   8,
         8,  84,  84, 106, 106,  66,  66,  68,  68,  34,  34,  22,  22,
        37,  37, 138, 138,  73,  73, 135, 135,  77,  77, 104, 104,  28,
        28, 134, 134,   5,   5, 116, 116, 121, 121, 139, 139,  17,  17,
        59,  59,  10,  10,  47,  47,  11,  11,  70,  70,  13,  13, 114,
       114,  91,  91, 102, 102,  81,  81,  18,  18,  86,  85,  71,  36,
        90,  31, 136, 136, 141, 141, 113, 113,  17,  99,  49,  41,  49,
        49,  83,  83,  30,  30,  17,  35,  38,  38, 103, 103,  64,  64,
        48,  96,  43,  43,  53,  53,  45,  21,  44,  44,  87,  36,  94,
        94,  65,  65,  26,  26,   1,  31, 107, 107, 111, 111,  25,  25,
       137, 137,  69, 105,  12,  12,  33,  33,  54,  54,  69,  69,   9,
         9,  75,  75,  60,  60,  20,  64, 110, 110, 108, 108,  57,  57,
        61,  61,  74,  74,  78,  78,  59,  85, 140, 140, 113, 124, 123,
       123,  23,  48,  72,  72,  24,  24,  67,  59,  85,  55,  7

In [26]:
accuracy_speaker = metrics.accuracy_score(y_test_speaker,y_pred_speaker)*100
accuracy_speaker

92.25352112676056

In [27]:
creport_speaker = classification_report(y_test_speaker, y_pred_speaker)
print(creport_speaker)

              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         2
          16       1.00      1.00      1.00         2
          17       0.40      1.00      0.57         2
          18       1.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
